# Tutorial

We are going to use the Volve data.
We want to read in the base and monitor surveys, calculate some simple 4D attributes (4D difference and NRMS) and apply a frequency filter to the data.

Start with importing some basic packages; numpy (as the data will be read into a numpy array), segyio to read the data, matlotlib to plot it and time so we can get some statistics on how long things take

***Do we need pandas?  (possible if we are reading the headers)***

In [ ]:
import numpy as np
import segyio
import matplotlib.pyplot as plt
import time

In [ ]:
# this should be a less bonkers path - local to the notebook is probably fine, or with a download link
base_segy = 'C:/Users/GDMA/SeisData/Volve/ST0202ZDC12-PZ-PSDM-KIRCH-FULL-T.MIG_FIN.POST_STACK.3D.JS-017534.segy'

In [ ]:
with segyio.open('base_sgy') as f:
    print(f.tracecount)

In [ ]:
with segyio.open('base_sgy', ignore_geometry = True) as f:
    for text in f.text:
        print(text)

In [ ]:
f = segyio.open('base_sgy', ignore_geometry = True)
ntraces    = len(f.trace)
inlines    = []
crosslines = []

for i in range(nlines):
    headeri = f.header[i]
    inlines.append(headeri[segyio.su.il])
    crosslines.append(headeri[segyio.su.xl])

print(f'{ntraces} traces')
print(f'first 10 inlines: {inlines[:10]}')
print(f'first 10 crosslines: {crosslines[:10]}')

In [ ]:
# ok, so what in/crosslines *do* we have?
# notice how they taper off
plt.plot(inlines, crosslines)

In [ ]:
# i only used to look up the header - can be done directly
# this applies to traces too, and, when segyio can figure it out,
# inlines and crosslines
for header in f.header:
    print(header[segyio.su.cdpx, segyio.su.cdpy])

### segyio

Jørgen to add

Get some info about our data
- Read the headers?
- Plot the x,y locations of the survey?
- .....

Try to read the file

In [ ]:
with segyio.open(base_segy) as segyf:
    n_traces = segyf.tracecount
    sample_rate = segyio.tools.dt(segyf)/1000
    n_samples = segyf.samples.size
    n_il = len(segyf.iline)

Highlight problem is that the segy is not a perfect cube (number of inlines * number of xlines = number of traces).<br>
There are a number of possible solutions to this. Here use the solution given in one of the segyio example notebooks
https://github.com/equinor/segyio-notebooks/blob/master/notebooks/pylops/01_seismic_inversion.ipynb

***Need to check/change the naming of the variables since we will be reading 2 volumes***

In [ ]:
# Read all the traces then reshape in numpy
t0 = time.time()
f = segyio.open(base_segy, ignore_geometry=True)
traces = segyio.collect(f.trace)[:]
ntraces, nt = traces.shape

t = f.samples[:]
il = f.attributes(segyio.TraceField.INLINE_3D)[:]
xl = f.attributes(segyio.TraceField.CROSSLINE_3D)[:]

## Define regular IL and XL axes

il_unique = np.unique(il)
xl_unique = np.unique(xl)

il_min, il_max = min(il_unique), max(il_unique)
xl_min, xl_max = min(xl_unique), max(xl_unique)

# Get line increment
dt = t[1] - t[0]
dil = min(np.unique(np.diff(il_unique)))
dxl = min(np.unique(np.diff(xl_unique)))

# Create grid and get number of inlines & xlines
ilines = np.arange(il_min, il_max + dil, dil)
xlines = np.arange(xl_min, xl_max + dxl, dxl)
nil, nxl = ilines.size, xlines.size

ilgrid, xlgrid = np.meshgrid(np.arange(nil),
                             np.arange(nxl),
                             indexing='ij')

# Look-up table
traces_indeces = np.full((nil, nxl), np.nan)
iils = (il - il_min) // dil
ixls = (xl - xl_min) // dxl
traces_indeces[iils, ixls] = np.arange(ntraces)
traces_available = np.logical_not(np.isnan(traces_indeces))

# Reorganize traces in regular grid
d = np.zeros((nil, nxl, nt))
d[ilgrid.ravel()[traces_available.ravel()],
  xlgrid.ravel()[traces_available.ravel()]] = traces
# Return the time (in seconds to do this)
sgy_r_time = time.time() - t0
print(f'segy file with {ntraces} traces ({nil} inlines, {nxl} xlines) indexed and read in {sgy_r_time} s')

Explain the outputs <br>
Data is in a 3D numpy array and our inline, crossline and twt are also in separate numpy arrays

In [ ]:
d.shape

In [ ]:
ilines.shape

In [ ]:
ilines

### Plot the data
Now we've read it, let's look at a single line

In [ ]:
# Plot
imgplot = plt.imshow(d[nil//2,:,:], cmap='gray', aspect='auto', vmin=-5, vmax=5)

That doesn't look right - we need to consider the origin and how the data is read.
Also add the correct labelling on the axis rather than simply the index

In [ ]:
# Plot correctly
# Line
extent = [xlines[0],xlines[-1],t[-1],t[0]]
imgplot = plt.imshow(d[nil//2,:,:].T, cmap='gray', aspect='auto', vmin=-5, vmax=5, extent=extent)

In [ ]:
# T-Slice
imgplot = plt.imshow(d[:,:,575], cmap='gray', origin='lower', aspect='auto', vmin=-5, vmax=5)

### Interactive plotting

Ideally we would like to be scan through some lines.
Import some new plotting packages.

In [ ]:
# do we need these two?
#import holoviews as hv
#from holoviews import opts
# Definitely need these
import xarray as xr
import hvplot.xarray
import panel as pn

In [ ]:
def plot_inl(inl):
    idx = inl - ilines[0]
    da = xr.DataArray(d[idx,:,:].T)    
    p = da.hvplot.image(clim=(-5, 5), cmap='gray', flip_yaxis=True) 
    return p

In [ ]:
il_slice = pn.interact(plot_inl, inl=ilines)
il_slice

### Xarray ?

Plot above again has no useful axis info <br>
Hvplot uses xarray - explain a bit more and put base into an xarray with coordinates - reference back to segysak

In [ ]:
#Create a Data Array
da = xr.DataArray(data=d,
                  dims=['il','xl','twt'],
                  coords={'il': ilines,
                          'xl': xlines,
                          'twt': t})

### Monitor survey Using seismic-zfp

Now we've read and reviewed one vintage, let's read the second

Another option to get around the irregular geometry would be to use segysak (ref Tony's tutorial).  Here we'll use a third option and convert the segy file into another format that automatically pads the data.  We will use seismic-zfp which forms a compressed format.  We won't go into detail on this format but more information can be found .... (add links)

In [ ]:
import seismic_zfp
from seismic_zfp.conversion import SegyConverter, SgzReader

First step is to convert the file

In [ ]:
monitor_segy = 'C:/Users/GDMA/SeisData/Volve/ST10010ZDC12-PZ-PSDM-KIRCH-FULL-T.MIG_FIN.POST_STACK.3D.JS-017534.segy'
monitor_sgz = 'C:/Users/gdma/git/volve/data/ST10010_4D_Monitor.sgz'

In [ ]:
sgz_t0 = time.time()
with SegyConverter(monitor_segy) as converter:
    # Create a "standard" SGZ file with 8:1 compression, using in-memory method
    converter.run(monitor_sgz, bits_per_voxel=4)
sgz_elapse = time.time() - sgz_t0
print(f'Converted to sgz in {sgz_elapse} s')

Now it's converted we can read it

In [ ]:
# Read the cube from zgy
t0 = time.time()
with SgzReader(monitor_sgz) as reader:
    n_traces_m = reader.tracecount
    n_il_m = reader.n_ilines
    n_xl_m = reader.n_xlines
    num_samples_m = reader.n_samples
    ilines_m = reader.ilines
    xlines_m = reader.xlines
    data_m = reader.read_volume()
zgy_r_time = time.time() - t0
print(f'sgz file with {n_traces_m} traces ({n_il_m} inlines, {n_xl_m} xlines) read in {zgy_r_time} s')

*Comment on run times - overhead of conversion v speed of reading, also comment on compression*

Now we've read the monitor survey, let's check they are both the same size and have the same inline, crossline range

In [ ]:
# Shape of arrays
data_m.shape
d.shape
# Line ranges
ilines.min()
ilines.max()


Plot a slice as a quick qc

In [ ]:
# Add monitor to xarray
# Create a dataset
volve_ds = da.to_dataset(name='base')

In [ ]:
# Add the monitor survey
monitor = xr.DataArray(data_m)
volve_ds['monitor'] = (['il','xl','twt'],monitor)

### 4D Analysis
Now we have the 2 volumes we can calculate some simple 4D attributes
- Calculate the 4D difference 
- Calculate the NRMS

### Frequency spectra and filtering
- Look at the frequency spectra
- Filter


In [ ]:
from scipy import signal

In [ ]:
# Freq spectra - make this a function as we'll use it again
S = np.mean(np.abs(np.fft.fft(d[:, :, 500:750], axis=-1)), axis=(0, 1))
freq = np.fft.fftfreq(len(S), d=0.004)
f, amp = freq[:S.size//2], np.abs(S[:S.size//2])

In [ ]:
plt.plot(f, amp, color='red')

In [ ]:
# Lowpass filter
# This is using filtfilt - better to use sosfiltfilt
fs = 1/0.004
nyq = fs / 2
cutoff = 0.25
b, a = signal.butter(5, cutoff, btype='lowpass')
d_filt = signal.filtfilt(b, a, d, axis=2)

In [ ]:
# Freq spectra again

In [ ]:
# Interactive plot with before, after and difference

In [ ]:
# Filter the monitor and calc new difference - depends on time?